<div align=center>
<font size=5>
<font/>
<p></p>
 <br/>

<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Linear Algebra
</font>
<br/>
<br/>

Spring 2023

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 1
            	<br/>
			</font>
</font>
                <br/>
    </div>

<div>
    <font color="orange" size=6>
    Q1
    </font>
    <hr/>
</div>
As you know, computers are quite "dumb" and constantly strive to be as unintelligent as possible. One example of their "dumbness" is their inability to understand texts and the similarities between them. Thus, in order to help computers understand a text, we have no choice but to produce a vector representation of it. A well-known vector representation of text in computers is the ASCII or UTF code, which is used to store characters and to store a text using a sequence of them. However, the problem with this representation is that it has no connection to the semantic dimension of words and sentences.

We aim to introduce a new representation for sentences. Suppose we have a collection of m sentences, which together consist of n distinct words. Now, using these words and sentences, we wish to define the following two matrices:

**(a) Word Frequency Matrix:** The more frequently a word appears in a sentence, the more likely it is to play a significant role in the meaning of that sentence. The word frequency matrix, which we denote by F, is an n × m matrix where the element in row i and column j is equal to the frequency of word j in sentence i. For example, in the sentence:

"I like eating dessert in the desert,"

we have 6 words, and the word "desert" appears twice. Therefore, the frequency of this word in the sentence is 2.

**(b) Word Commonality Matrix:** The more frequently a word appears across many sentences, the less semantic weight it is likely to carry. For instance, prepositions and conjunctions appear in the vast majority of sentences. The word commonality matrix (or vector) is an n × 1 matrix where the element in row i is the logarithm of the ratio of the total number of sentences to the number of sentences that contain word i. Formally, if D is the set of all sentences and $D_i$ is the subset of sentences that contain word i, the element in row i of the word commonality matrix, which we denote by I, is given by:
$$I[i] = ln \frac{|D|}{|D_i|}
$$
Now, the representation of sentence i is considered to be the element-wise product of row i of the frequency matrix and the word commonality vector.

In this problem, we are given a set called S, which contains several sentences. The first step is to compute the representations of these sentences. Then, we are given a sentence query for which its representation must be computed in the same way as for the sentences above. Finally, we need to return the index of the sentence in S that is most similar to the query. This similarity is determined by calculating the angle between their representations.

Before performing any of these steps, punctuation must be removed from the text, and letter case should be ignored (i.e., treat upper and lower case letters as the same).

**Input:** The first line of the input contains an integer n, which represents the size of the set |S|. In the next n lines, each line contains a sentence from the set S. Finally, the last line contains the query sentence.

**Output:** The output should be the index of the sentence that has the highest similarity to the query.

In [1]:
import math
import string
from collections import Counter
import numpy as np

In [2]:
def preprocess(sentence):
    sentence = sentence.lower().translate(str.maketrans('', '', string.punctuation))
    return sentence.split()

In [3]:
def compute_word_frequencies(sentences):
    word_list = set()
    word_frequencies = []
    
    for sentence in sentences:
        words = preprocess(sentence)
        word_count = Counter(words)
        word_list.update(words)
        word_frequencies.append(word_count)
    
    return list(word_list), word_frequencies

In [4]:
def build_matrices(word_list, word_frequencies, sentences):
    n = len(word_list)
    m = len(sentences)
    word_index = {word: i for i, word in enumerate(word_list)}
    
    F = np.zeros((m, n))
    I = np.zeros(n)
    
    for j, word in enumerate(word_list):
        word_count_in_sentences = sum(1 for freq in word_frequencies if word in freq)
        I[j] = math.log(m / word_count_in_sentences) if word_count_in_sentences > 0 else 0
    
    for i, freq in enumerate(word_frequencies):
        for word, count in freq.items():
            F[i, word_index[word]] = count
            
    return F, I

def sentence_representation(F, I):
    return F * I

In [5]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return 0
    return dot_product / (magnitude_v1 * magnitude_v2)

def find_most_similar_sentence(S):
    sentences = S[:-1]
    query_sentence = S[-1]
    word_list, word_frequencies = compute_word_frequencies(sentences)
    query_frequency = Counter(preprocess(query_sentence))
    F, I = build_matrices(word_list, word_frequencies, sentences)
    sentence_vectors = sentence_representation(F, I)
    query_vector = np.zeros(len(word_list))
    word_index = {word: i for i, word in enumerate(word_list)}
    
    for word, count in query_frequency.items():
        if word in word_index:
            idx = word_index[word]
            query_vector[idx] = count * I[idx]
    
    similarities = [cosine_similarity(sentence_vectors[i], query_vector) for i in range(len(sentences))]
    
    return np.argmax(similarities) + 1

In [6]:
def similarity(test_input):
    test_case = test_input.strip().split('\n')
    n = int(test_case[0])
    S = test_case[1:]
    most_similar_sentence_index = find_most_similar_sentence(S)
    print(most_similar_sentence_index)


Let's test the code

In [7]:
test_input = '''
3
I like eating dessert in the desert
The sky is clear today
I will go to the desert tomorrow
desert in summer is hot
'''
similarity(test_input) # The output should be 1


test_input = '''
10
evening? majority? relinquish harmful, jump. nap score. monstrous? jump. disposition? bulb greet bulb? displace 
jump jacket. suite. motivation cultural. the. care? council? flag? 
mosquito, tiptoe. job issue. provision leaflet. particle job, identity. horn, architecture evening creation 
vegetable, stadium, bury? chase? chase. disposition. 
gallery range? layout, able? cane, earthflax. delicate? productive, 
gutter. night. representative, impound. qualification, 
confusion, displace sunshine? put. choke. feature, relinquish cultural dominate. attraction? superior, determine? bounce? tiger, 
gallery, argument magnitude, score layout. confine? impound? diamond, qualification is? identity 
snail, preach, simplicity paragraph. marsh, circulation, gutter, anniversary, attraction, evening? is 
issue. circumstance? able jump. eyebrow, 
evening. volcano? category. trivial. fuel morale. 
'''
similarity(test_input) # The output should be 9

1
9
